In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mobile-phone-specifications-and-prices/ndtv_data_final.csv


In [43]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import ipywidgets as widgets

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ipywidgets import interact, widgets
from IPython.display import display, HTML
from scipy import stats

In [44]:
data = pd.read_csv('/kaggle/input/mobile-phone-specifications-and-prices/ndtv_data_final.csv')

In [45]:
# Define sample reviews with corresponding ratings
reviews_with_ratings = {
    "Excellent performance, worth the price.": 5,
    "Great battery life, perfect for gaming.": 5,
    "Amazing camera quality, a bit expensive.": 4,
    "Good value for money, smooth performance.": 4,
    "Battery could be better, decent overall.": 3,
    "Highly recommend for photography lovers.": 5,
    "Affordable and reliable, good screen quality.": 4,
    "Sleek design, average battery life.": 3,
    "Perfect for everyday use, budget-friendly.": 4,
    "Premium quality, outstanding features.": 5
}

# Expand reviews to match the dataset size
reviews = list(reviews_with_ratings.keys())
expanded_reviews = reviews * (len(data) // len(reviews)) + reviews[:len(data) % len(reviews)]

# Assign ratings based on the review content
ratings = [reviews_with_ratings[review] for review in expanded_reviews]

# Add the 'Customer Reviews' and 'Ratings' columns to the dataset
data['Customer Reviews'] = expanded_reviews
data['Ratings'] = ratings


In [46]:

# Fill missing values
data['Ratings'] = data['Ratings'].fillna(data['Ratings'].mean())  # Use mean for ratings
data['Price'] = data['Price'].fillna(data['Price'].mean())  # Use mean for price
data = data.fillna('Unknown')  # Fill other categorical columns with 'Unknown'


In [47]:
headers = data.columns.tolist()
print(headers)

['Unnamed: 0', 'Name', 'Brand', 'Model', 'Battery capacity (mAh)', 'Screen size (inches)', 'Touchscreen', 'Resolution x', 'Resolution y', 'Processor', 'RAM (MB)', 'Internal storage (GB)', 'Rear camera', 'Front camera', 'Operating system', 'Wi-Fi', 'Bluetooth', 'GPS', 'Number of SIMs', '3G', '4G/ LTE', 'Price', 'Customer Reviews', 'Ratings']


In [48]:
# Important Feature Selection
selected = ['Name', 'Brand', 'Model', 'Battery capacity (mAh)','Touchscreen', 'Processor', 'RAM (MB)', 
            'Internal storage (GB)', 'Rear camera', 'Front camera', 'Operating system', 'Wi-Fi', 'Bluetooth', 
            'GPS', 'Number of SIMs', 'Price', 'Customer Reviews','Ratings']

In [49]:
# Replacing the missing values with null string
for feature in selected:
    data[feature] = data[feature].fillna('') 

In [50]:
# Handle outliers in the 'Price' column
price_summary = data['Price'].describe()
iqr = price_summary['75%'] - price_summary['25%']
lower_bound = price_summary['25%'] - 1.5 * iqr
upper_bound = price_summary['75%'] + 1.5 * iqr
cleaned_data = data[(data['Price'] >= lower_bound) & (data['Price'] <= upper_bound)]

In [51]:
# Ensuring numeric columns are converted to strings before concatenation
combined_alf = (data['Name'].astype(str) + ' ' +
                data['Brand'].astype(str) + ' ' +
                data['Model'].astype(str) + ' ' +
                data['Touchscreen'].astype(str) + ' ' +
                data['Processor'].astype(str) + ' ' +
                data['Operating system'].astype(str) + ' ' +
                data['Wi-Fi'].astype(str) + ' ' +
                data['Bluetooth'].astype(str) + ' ' +
                data['GPS'].astype(str) + ' ' +
                data['Customer Reviews'].astype(str))

combined_num = (data['Battery capacity (mAh)'].astype(str) + ' ' +
                data['RAM (MB)'].astype(str) + ' ' +
                data['Internal storage (GB)'].astype(str) + ' ' +
                data['Front camera'].astype(str) + ' ' +
                data['Rear camera'].astype(str) + ' ' +
                data['Number of SIMs'].astype(str) + ' ' +
                data['Price'].astype(str)+ ' ' +
                data['Ratings'].astype(str))

combined_features = combined_alf + ' ' + combined_num

In [52]:
print(combined_features)

0       OnePlus 7T Pro McLaren Edition OnePlus 7T Pro ...
1       Realme X2 Pro Realme X2 Pro Yes 8 Android Yes ...
2       iPhone 11 Pro Max Apple iPhone 11 Pro Max Yes ...
3       iPhone 11 Apple iPhone 11 Yes 6 iOS Yes Yes Ye...
4       LG G8X ThinQ LG G8X ThinQ Yes 8 Android Yes Ye...
                              ...                        
1354    Intex Aqua A2 Intex Aqua A2 Yes 4 Android Yes ...
1355    Videocon Infinium Z51 Nova+ Videocon Infinium ...
1356    Intex Aqua Y4 Intex Aqua Y4 Yes 2 Android Yes ...
1357    iBall Andi4 B20 iBall Andi4 B20 Yes 1 Android ...
1358    iBall Andi Avonte 5 iBall Andi Avonte 5 Yes 4 ...
Length: 1359, dtype: object


In [53]:
# Converting the text data to feature vectores
vectorizer = TfidfVectorizer()
feature_vector = vectorizer.fit_transform(combined_features)
print(feature_vector)

  (0, 648)	0.23247174282058186
  (0, 546)	0.13988805350125116
  (0, 118)	0.05604109371410968
  (0, 257)	0.16456608935857933
  (0, 44)	0.17709706748713558
  (0, 474)	0.21994076469202564
  (0, 1400)	0.10184062276654901
  (0, 1533)	0.10184062276654901
  (0, 1602)	0.10184062276654901
  (0, 1379)	0.08053177317388599
  (0, 1086)	0.10184062276654901
  (0, 971)	0.03229965058476271
  (0, 1670)	0.12362077898168641
  (0, 1074)	0.4220997884362737
  (0, 1289)	0.4649434856411637
  (0, 1405)	0.24010286830197528
  (0, 811)	0.4220997884362737
  (0, 1336)	0.34924666403706817
  (1, 294)	0.2920054936379016
  (1, 702)	0.2662195006642531
  (1, 677)	0.17472053386459896
  (1, 462)	0.10858094790779405
  (1, 1132)	0.14566493769422734
  (1, 1115)	0.08462729180425588
  (1, 1378)	0.11518640993260748
  :	:
  (1357, 50)	0.25393238835295323
  (1357, 1172)	0.39301026110973586
  (1357, 591)	0.1363317826370033
  (1357, 981)	0.11757998844222244
  (1357, 1051)	0.11757998844222244
  (1357, 1489)	0.11757998844222244
  (1357

In [54]:
# getting similarity scores using Cosine Similarity
similarity = cosine_similarity(feature_vector)
print(similarity)

[[1.         0.11029983 0.08680864 ... 0.01849039 0.05011593 0.02124428]
 [0.11029983 1.         0.16346478 ... 0.02644723 0.04528372 0.04897308]
 [0.08680864 0.16346478 1.         ... 0.02833818 0.02100116 0.02367064]
 ...
 [0.01849039 0.02644723 0.02833818 ... 1.         0.04515206 0.02406165]
 [0.05011593 0.04528372 0.02100116 ... 0.04515206 1.         0.19861789]
 [0.02124428 0.04897308 0.02367064 ... 0.02406165 0.19861789 1.        ]]


In [55]:
print(similarity.shape)

(1359, 1359)


In [56]:
# input from user
name = input('Enter your favourite mobile name =  ')

Enter your favourite mobile name =   nokia


In [57]:
# Creating the list of the mobile phone names in the dataset

phone_list = data['Brand'].tolist()
print(phone_list)

['OnePlus', 'Realme', 'Apple', 'Apple', 'LG', 'OnePlus', 'OnePlus', 'Samsung', 'Asus', 'Xiaomi', 'Oppo', 'Realme', 'Xiaomi', 'OnePlus', 'Oppo', 'Realme', 'Huawei', 'Xiaomi', 'Huawei', 'LG', 'OnePlus', 'Apple', 'Apple', 'Apple', 'Google', 'Google', 'Asus', 'Samsung', 'LG', 'Asus', 'Huawei', 'OnePlus', 'Asus', 'Xiaomi', 'Nokia', 'Samsung', 'Samsung', 'HTC', 'Samsung', 'Apple', 'Apple', 'Apple', 'Motorola', 'HTC', 'Nokia', 'Samsung', 'Samsung', 'LG', 'OnePlus', 'Samsung', 'Google', 'Apple', 'Apple', 'Honor', 'OnePlus', 'Asus', 'Xiaomi', 'Samsung', 'Samsung', 'Motorola', 'Huawei', 'Apple', 'Yu', 'Samsung', 'Samsung', 'Yu', 'Xiaomi', 'Realme', 'Poco', 'Samsung', 'Realme', 'Samsung', 'Vivo', 'Vivo', 'Realme', 'Realme', 'Vivo', 'Nubia', 'Samsung', 'Vivo', 'Oppo', 'Realme', 'Xiaomi', 'Nokia', 'Samsung', 'Realme', 'Realme', 'Vivo', 'Xiaomi', 'Asus', 'Samsung', 'Xiaomi', 'Motorola', 'Nubia', 'Oppo', 'Google', 'Samsung', 'Black Shark', 'OnePlus', 'LG', 'Vivo', 'Xiaomi', 'Xiaomi', 'Samsung', 'Sams

In [58]:
# finding the closest match
import difflib
closest_match = difflib.get_close_matches(name, phone_list)
print(closest_match)

['Nokia', 'Nokia', 'Nokia']


In [59]:
close_match = closest_match[0]
print(close_match)

Nokia


In [60]:
# Finding the index of the mobile
matching_indices = data[data.Brand == close_match]['Unnamed: 0'].values[0]
print(matching_indices)

34


In [61]:
similarity_score = list(enumerate(similarity[matching_indices]))
print(similarity_score)

[(0, 0.09039481816399164), (1, 0.12000955830837118), (2, 0.11109522914862044), (3, 0.1257322490247774), (4, 0.20258554732882358), (5, 0.12481464589074483), (6, 0.0449088108226297), (7, 0.14737093207731006), (8, 0.036675044102098125), (9, 0.05137721962659482), (10, 0.08121162886952941), (11, 0.09935713177886832), (12, 0.0750473254187076), (13, 0.07520849293630336), (14, 0.14523158688866925), (15, 0.1005788339252024), (16, 0.02736726229058666), (17, 0.11758164009268174), (18, 0.03480677618579927), (19, 0.06383800270895179), (20, 0.05533799325997741), (21, 0.08815916352656328), (22, 0.08677777324282085), (23, 0.09247936594472204), (24, 0.229273510311337), (25, 0.1159229287830342), (26, 0.078104227966456), (27, 0.11848525884782629), (28, 0.04667367519577025), (29, 0.036503331884271566), (30, 0.08500841893332613), (31, 0.12031016789466875), (32, 0.08631316773410355), (33, 0.1338718192542749), (34, 1.0), (35, 0.10037078735722378), (36, 0.11704176745966224), (37, 0.08547776952752176), (38, 0.

In [62]:
# Sorting the movies based on their similarity score
sorted_similar_phones = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_phones)

[(34, 1.0), (123, 0.5988127015586031), (134, 0.577460791033686), (287, 0.5587356297041087), (44, 0.5552534351100488), (504, 0.5459393747291378), (120, 0.5438501089121752), (959, 0.43397180156891507), (113, 0.4313834011263109), (110, 0.4280724009258487), (347, 0.417799427001127), (466, 0.40442143311882317), (809, 0.398899659808155), (465, 0.39542573461728064), (774, 0.3951892783002161), (1174, 0.39371950868895267), (620, 0.37661832062527756), (481, 0.3760032656300903), (174, 0.36972617743658237), (404, 0.3685911188083259), (83, 0.36778162165078365), (754, 0.3604826603408405), (718, 0.35979974961907124), (444, 0.34833660018928714), (904, 0.3325327534359612), (991, 0.3091092016085989), (907, 0.29284498844576623), (728, 0.29148742885014806), (54, 0.2892116081736781), (114, 0.2859059875074634), (751, 0.28580670346687015), (1261, 0.2852126390323247), (132, 0.2841515653502467), (163, 0.2778033968917432), (624, 0.27021208896328086), (204, 0.2683648786597872), (416, 0.26488142079711646), (367, 

In [63]:
from tabulate import tabulate

# Assuming sorted_similar_phones is already defined and contains the necessary phone indices

table_data = []
i = 1
for phones in sorted_similar_phones:
    index = phones[0]
    brand_index = data[data['Unnamed: 0'] == index]['Brand'].values[0]
    name_index = data[data['Unnamed: 0'] == index]['Name'].values[0]
    model_index = data[data['Unnamed: 0'] == index]['Model'].values[0]
    cap_index = data[data['Unnamed: 0'] == index]['Battery capacity (mAh)'].values[0]
    ram_index = data[data['Unnamed: 0'] == index]['RAM (MB)'].values[0]
    storage_index = data[data['Unnamed: 0'] == index]['Internal storage (GB)'].values[0]
    price_index = data[data['Unnamed: 0'] == index]['Price'].values[0]
    review_index = data[data['Unnamed: 0'] == index]['Customer Reviews'].values[0]
    
    if i <= 50:
        table_data.append([i, brand_index, name_index, model_index, cap_index, ram_index, storage_index, price_index, review_index])
        i += 1

# Define the headers
headers = ["#", "Brand", "Name", "Model", "Battery (mAh)", "RAM (MB)", "Storage (GB)", "Price", "Customer Reviews"]

# Print the nicely formatted table
print("Phones Suggested:")
print(tabulate(table_data, headers=headers, tablefmt="fancy_grid"))


Phones Suggested:
╒═════╤══════════╤════════════════════════════╤════════════════════╤═════════════════╤════════════╤════════════════╤═════════╤═══════════════════════════════════════════════╕
│   # │ Brand    │ Name                       │ Model              │   Battery (mAh) │   RAM (MB) │   Storage (GB) │   Price │ Customer Reviews                              │
╞═════╪══════════╪════════════════════════════╪════════════════════╪═════════════════╪════════════╪════════════════╪═════════╪═══════════════════════════════════════════════╡
│   1 │ Nokia    │ Nokia 7 Plus               │ 7 Plus             │            3800 │       4000 │             64 │   12000 │ Battery could be better, decent overall.      │
├─────┼──────────┼────────────────────────────┼────────────────────┼─────────────────┼────────────┼────────────────┼─────────┼───────────────────────────────────────────────┤
│   2 │ Nokia    │ Nokia 6.1 Plus             │ 6.1 Plus           │            3060 │       4000 │        

In [69]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np

# Assuming 'data' is your DataFrame
X = data.drop(['Ratings'], axis=1)
y = data['Ratings']

# Separate numeric and categorical columns
numeric_cols = X.select_dtypes(include=[np.number]).columns
categorical_cols = X.select_dtypes(exclude=[np.number]).columns

# Preprocessing pipeline for numeric columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
])

# Preprocessing pipeline for categorical columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

# Combine both transformers in a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with preprocessing and the RandomForestRegressor model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])


In [70]:
# Hyperparameter tuning using GridSearchCV
param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best model from GridSearchCV
best_model = grid_search.best_estimator_

# Evaluate the model
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test) # Calculate accuracy and error metrics
print("Training Mean Squared Error:", mean_squared_error(y_train, y_pred_train))
print("Testing Mean Squared Error:", mean_squared_error(y_test, y_pred_test))
print("Training R-squared:", r2_score(y_train, y_pred_train))
print("Testing R-squared:", r2_score(y_test, y_pred_test))

# Check for overfitting/underfitting by comparing training and testing accuracy
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

print(f"Training Accuracy: {train_accuracy}")
print(f"Testing Accuracy: {test_accuracy}")

# Cross-validation to check for overfitting/underfitting
cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='r2')
print("Cross-validated R-squared scores:", cv_scores)
print("Mean Cross-validated R-squared:", cv_scores.mean())

Training Mean Squared Error: 0.0
Testing Mean Squared Error: 0.0
Training R-squared: 1.0
Testing R-squared: 1.0
Training Accuracy: 1.0
Testing Accuracy: 1.0
Cross-validated R-squared scores: [1. 1. 1. 1. 1.]
Mean Cross-validated R-squared: 1.0


In [72]:
# Get the best model from GridSearchCV
best_model = grid_search.best_estimator_

# Evaluate the model
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

# Calculate metrics
train_mse = mean_squared_error(y_train, y_pred_train)
test_mse = mean_squared_error(y_test, y_pred_test)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

# Output results
print("Training Mean Squared Error:", train_mse)
print("Testing Mean Squared Error:", test_mse)
print("Training R-squared:", train_r2)
print("Testing R-squared:", test_r2)

# Cross-validation to evaluate generalization
cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='r2')
print("Cross-validated R-squared scores:", cv_scores)
print("Mean Cross-validated R-squared:", cv_scores.mean())

# Check if the model is overfitting/underfitting
if train_r2 > test_r2 + 0.1:
    print("Model is likely overfitting. Consider regularization or simplifying the model.")
elif test_r2 < 0.7:
    print("Model is underfitting. Consider adding features or tuning hyperparameters.")
else:
    print("Model is well-fitted.")

Training Mean Squared Error: 0.0
Testing Mean Squared Error: 0.0
Training R-squared: 1.0
Testing R-squared: 1.0
Cross-validated R-squared scores: [1. 1. 1. 1. 1.]
Mean Cross-validated R-squared: 1.0
Model is well-fitted.


In [71]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interactive
from tabulate import tabulate

# Define the selected columns based on the dataset
selected = [
    'Name', 'Brand', 'Model', 'Battery capacity (mAh)', 'Touchscreen', 'Processor',
    'RAM (MB)', 'Internal storage (GB)', 'Rear camera', 'Front camera',
    'Operating system', 'Wi-Fi', 'Bluetooth', 'GPS', 'Number of SIMs',
    'Price', 'Customer Reviews', 'Ratings'
]

# Filter the data based on selected features (check for missing columns)
existing_columns = [col for col in selected if col in data.columns]
filtered_data = data[existing_columns]

# Sort the data alphabetically by 'Name' by default
sorted_data = filtered_data.sort_values(by='Name')

# Function to display filtered and sorted data in a table
def display_table(filtered_data):
    table_data = []
    for i, phone in filtered_data.iterrows():
        table_data.append([
            i + 1,
            phone.get('Brand', ''),
            phone.get('Name', ''),
            phone.get('Model', ''),
            phone.get('Battery capacity (mAh)', ''),
            phone.get('RAM (MB)', ''),
            phone.get('Internal storage (GB)', ''),
            phone.get('Price', ''),
            phone.get('Customer Reviews', ''),
            phone.get('Ratings', ''),
        ])
    
    headers = ["#", "Brand", "Name", "Model", "Battery (mAh)", "RAM (MB)", "Storage (GB)", "Price", "Customer Reviews", "Ratings"]
    print(tabulate(table_data, headers=headers, tablefmt="fancy_grid"))

# Function to filter data based on widget inputs
def filter_data(brand, price_range, rating_range, processor):
    filtered = sorted_data
    
    # Filter by Brand
    if brand != "All Brands":
        filtered = filtered[filtered['Brand'] == brand]
    
    # Filter by Price Range
    min_price, max_price = price_range
    filtered = filtered[(filtered['Price'] >= min_price) & (filtered['Price'] <= max_price)]
    
    # Filter by Ratings Range
    min_rating, max_rating = rating_range
    filtered = filtered[(filtered['Ratings'] >= min_rating) & (filtered['Ratings'] <= max_rating)]
    
    # Filter by Processor
    if processor != "All Processors":
        filtered = filtered[filtered['Processor'] == processor]
    
    # Display the filtered data in the table
    display_table(filtered)

# Widgets for filtering
brand_widget = widgets.Dropdown(
    options=['All Brands'] + sorted_data['Brand'].dropna().unique().tolist(),
    description='Brand:',
    value='All Brands'
)

price_widget = widgets.IntRangeSlider(
    value=[int(sorted_data['Price'].min()), int(sorted_data['Price'].max())],
    min=int(sorted_data['Price'].min()),
    max=int(sorted_data['Price'].max()),
    step=500,
    description='Price Range:',
    continuous_update=False
)

rating_widget = widgets.FloatRangeSlider(
    value=[float(sorted_data['Ratings'].min()), float(sorted_data['Ratings'].max())],
    min=float(sorted_data['Ratings'].min()),
    max=float(sorted_data['Ratings'].max()),
    step=0.1,
    description='Ratings Range:',
    continuous_update=False
)

processor_widget = widgets.Dropdown(
    options=['All Processors'] + sorted_data['Processor'].dropna().unique().tolist(),
    description='Processor:',
    value='All Processors'
)

# Interactive plot with widgets
interactive_plot = interactive(
    filter_data, 
    brand=brand_widget, 
    price_range=price_widget, 
    rating_range=rating_widget,
    processor=processor_widget
)

# Display the interactive plot
interactive_plot


interactive(children=(Dropdown(description='Brand:', options=('All Brands', '10.or', 'Acer', 'Alcatel', 'Apple…